In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt

%matplotlib tk

In [2]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [9]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

xminnoise = int(np.round(11114*(fs/1000)))
xmaxnoise = int(np.round(18511*(fs/1000)))

noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

xminspike = int(np.round(130826*(fs/1000)))
xmaxspike = int(np.round(131699*(fs/1000)))

spike_data = filtereddf.iloc[xminspike:xmaxspike,0]

In [4]:
#threshold

def test_valid_window(window, test_level = 5):
    """
    window : the window in the signal that has to be tested
    
    This funtion test the window to insure that it doesn't contain the signal of interest (spike)
    """
    #non zero ?
    second = np.percentile(window, 2)
    thirtyth = np.percentile(window, 30)
    #print(str(second) + "\t" + str(thirtyth) + "\t" + str(second/thirtyth))
    if abs(second/thirtyth) < test_level : 
        return True
    else : 
        return False
    

def init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5):
    
    nb_valid_windows = 0
    list_RMS = []
    noise_levels = []
     
    #boucle en indice
#    for window_index in range(0,len(signal)-(len(signal)%int(fs*noise_window_size)),int(fs*noise_window_size)):
    for window_index in range(0,len(signal),int(fs*noise_window_size)):
        test = test_valid_window(signal.iloc[window_index: window_index + int(fs*noise_window_size)], test_level)
        if nb_valid_windows < required_valid_windows:
            if test == True :
                RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                nb_valid_windows += 1
            
            if nb_valid_windows == required_valid_windows:
                noise_level = np.percentile(list_RMS, 25)
                for elm in range(0, window_index, int(fs*noise_window_size)):
                    noise_levels.append(noise_level)
                
        else :
            """if test == True:
                if (window + int(fs*noise_window_size)) > (len(signal)-1) :
                    N25 = np.percentile(abs(signal.iloc[window:]), 25)
                else :
                    N25 = np.percentile(abs(signal.iloc[window: window + int(fs*noise_window_size)]), 25)
                noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
            noise_levels.append(noise_level)"""
            if test == True:
                if (window_index + int(fs*noise_window_size)) > (len(signal)-1) :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index:]**2))
                else :
                    RMS = np.sqrt(np.mean(signal.iloc[window_index: window_index + int(fs*noise_window_size)]**2))
                list_RMS.append(RMS)
                N25 = np.percentile(list_RMS, 25)
                new_noise_level = old_noise_level_propagation*noise_level + (1-old_noise_level_propagation)*N25
                noise_level = new_noise_level
            noise_levels.append(noise_level)
    
    noise_levels.append(noise_level)        
    plt.figure()
    plt.plot(list_RMS)
    plt.xlabel('Time Windows')
    plt.title('RMS values')
    plt.grid(True)
                
    return noise_levels

In [5]:
#find spike

def find_spike(signal, initial_index, noise_levels, fs, spike_centers, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    if initial_index < len(signal):
        i = initial_index
        for value in signal.iloc[initial_index:]:
            if value > threshold_factor*noise_levels[int((i/fs)//noise_window_size)]:
                while(True):
                    if i > initial_index + window_size*fs:
                        b_point = int(i - window_size*fs)
                    else :
                        b_point = initial_index
                    if i < len(signal)-window_size*fs-1:
                        e_point = int(i + window_size*fs)
                    else :
                        e_point = len(signal)-1

                    highest_value = signal.iloc[b_point: e_point].max()
                    #print("highest value:"+str(highest_value))
                    
                    if highest_value == value : # && 50% amplitude
                        #print("Found high spike")
                        #print("index:"+str(i))
                        spike_centers.append(i)
                        return e_point
                    
                    else:
                        #i = signal.index.get_loc(highest)
                        #print("b_point: " + str(b_point))
                        #print("e_point: " + str(e_point))
                        i = int(np.round(signal.iloc[b_point: e_point].idxmax()*fs/1000))#index(highest_value)#+b_pointà
                       #print("i: "+str(i))
                        value = signal.iloc[i]
                        
                break
                    
            if value < -threshold_factor*noise_levels[int((i/fs)//noise_window_size)]:
                while(True):
                    if i > initial_index + window_size*fs:
                        b_point = int(i - window_size*fs)
                    else :
                        b_point = initial_index
                    if i < len(signal)-window_size*fs-1:
                        e_point = int(i + window_size*fs)
                    else :
                        e_point = len(signal)-1

                    lowest_value = min(signal.iloc[b_point: e_point])
                    #print("lowest value:"+str(lowest_value))
                    #print("value: "+ str(value))

                    if lowest_value == value : # && 50% amplitude
                        #print("Found low spike")
                        #print("index:"+str(i))
                        spike_centers.append(i)                        
                        return e_point
                    
                    else:
                        #i = signal.index.get_loc(lowest)
                        #print("b_point: " + str(b_point))
                        #print("e_point: " + str(e_point))
                        i = int(np.round(signal.iloc[b_point: e_point].idxmin()*fs/1000))#Index.get_loc(lowest_value)#+b_point
                        #print("i: "+str(i))
                        value = signal.iloc[i]
                        
                break
                  
            i += 1
    return -2
            
def find_spikes(signal, noise_levels, fs, 
               window_size = 0.001, 
               noise_window_size = 0.01,
               threshold_factor = 4):
    
    initial = 0
    spike_centers = []
    
    while initial != -2:
        initial = find_spike(signal, initial, noise_levels, fs, spike_centers,
                             window_size = window_size, 
                             noise_window_size = noise_window_size,
                             threshold_factor = threshold_factor)
    
    return spike_centers

In [6]:
#record spike

def record_spikes(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002):
    
    data = np.array([[float(x) for x in range(int(np.round(fs*(t_before+t_after)))+1)]])
    
    for center in spike_centers:
        if center < int(np.round(fs*t_before)):
            spike = [0 for i in range(0, int(np.round(fs*t_before-center)))]
            #print('spike before concat:'+ str(len(spike)))
            #spike = np.concatenate(spike, signal.iloc[:center + int(np.round(fs*t_after))].values())
            spike = np.concatenate(spike, signal.values[:center + int(np.round(fs*t_after))])
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > int(np.round(len(signal)-fs*t_after)):
            #spike = signal.iloc[int(np.round(center-fs*t_before)):].values()
            spike = signal.values[int(np.round(center-fs*t_before)):]
            #print('spike before concat:'+str(len(spike)))
            spike = np.concatenate(((spike,[0 for i in range(0, int(np.round(fs*t_after-(len(signal)-center)))+1)])))
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            #spike = signal.iloc[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1].values()
            spike = signal.values[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]
            #print(spike)
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    #print(data)
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [10]:
################################################################################################
####   TEST ADABANDFLT AVEC SIGNAL ORIGINAL (PASSE-BANDE+WIENER)

# Choices:
#  - full_signal : entire signal from first electrode
#  - signal_filtered : entire signal from first electrode after bandpass
#  - noise_data : part of signal where only noise (after bandpass)
#  - spike_data : part of signal where burst (after bandpass)


signal = signal_filtered

In [11]:
fs = 25000

noise_levels = init_noise_levels(signal, fs, 
                      noise_window_size = 0.01,
                      required_valid_windows = 100,
                      old_noise_level_propagation = 0.8, 
                      test_level = 5)

In [12]:
plt.figure()
plt.plot(noise_levels)
plt.grid(True)
plt.xlabel('Time')
plt.ylabel('Noise Amplitude [µV]')
plt.title('Noise Levels')

Text(0.5, 1.0, 'Noise Levels')

In [13]:
spike_centers = find_spikes(signal, noise_levels, fs, 
                           window_size = 0.001, 
                           noise_window_size = 0.01,
                           threshold_factor = 4)

spike_centers

[1791,
 12616,
 14304,
 18690,
 18741,
 18782,
 18854,
 18908,
 18933,
 18961,
 18997,
 19025,
 19053,
 19079,
 19123,
 19151,
 19200,
 19240,
 19293,
 19318,
 19347,
 19389,
 19415,
 19450,
 19475,
 19501,
 19567,
 19597,
 19648,
 19697,
 19758,
 19835,
 19867,
 19896,
 19950,
 19975,
 20000,
 20056,
 20098,
 20153,
 20184,
 20262,
 20296,
 20344,
 20393,
 20428,
 20470,
 20518,
 20575,
 20604,
 20637,
 20687,
 20719,
 20756,
 20819,
 20844,
 20873,
 20945,
 20977,
 21002,
 21030,
 21093,
 21132,
 21190,
 21231,
 21263,
 21304,
 21330,
 21359,
 21431,
 21465,
 21520,
 21548,
 21575,
 21609,
 21723,
 21783,
 21808,
 21841,
 21903,
 21942,
 21980,
 22051,
 22100,
 22146,
 22192,
 22217,
 22249,
 22292,
 22317,
 22363,
 22388,
 22432,
 22465,
 22569,
 22598,
 22624,
 22698,
 22757,
 22824,
 22852,
 22895,
 22949,
 22987,
 23018,
 23043,
 23102,
 23174,
 23267,
 23292,
 23406,
 23873,
 27648,
 72423,
 72448,
 72482,
 72521,
 72546,
 72580,
 72632,
 72665,
 72699,
 72741,
 72781,
 72806,
 

In [ ]:
spike_data = record_spikes(signal, fs, spike_centers, 
                  t_before = 0.001, 
                  t_after = 0.002)

In [ ]:
plt.figure(figsize = (100,20))
#plt.plot(spike_data)
spike_data.iloc[:,:20].plot()
plt.xlabel('Time')
plt.grid()

In [ ]:
def record_spikes_oneline(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002):
    
    data = np.array(['NaN' for x in range(len(signal))])
    data = data.astype(float)
    times = np.array(['NaN' for x in range(len(signal))])
    times = times.astype(pd.Timestamp)
    
    for center in spike_centers:
        if center < int(np.round(fs*t_before)):
            data[:center + int(np.round(fs*t_after))] = signal.values[:center + int(np.round(fs*t_after))]
            times[:center + int(np.round(fs*t_after))] = signal.index[:center + int(np.round(fs*t_after))]
            
        elif center > int(np.round(len(signal)-fs*t_after)):
            data[int(np.round(center-fs*t_before)):] = signal.values[int(np.round(center-fs*t_before)):]
            times[int(np.round(center-fs*t_before)):] = signal.index[int(np.round(center-fs*t_before)):]
            
        else :
            data[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1] = signal.values[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]
            times[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1] = signal.index[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1]

    spike_data_oneline = pd.DataFrame(data, index = times.astype(float))
    
    return spike_data_oneline

In [ ]:
spike_data_oneline = record_spikes_oneline(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002)

In [ ]:
#signal.plot(color = 'blue')
#spike_data_oneline.plot(color = 'red')
plt.plot(df.index, signal, color = 'blue')
plt.plot(spike_data_oneline.index, spike_data_oneline, color = 'red')
plt.title('Filtered Signal with Detected Spikes with RMS')
plt.xlabel('Time Windows')
plt.ylabel('Amplitude [µV]')
plt.legend()
plt.grid(True)